# Zip folder unpacking script
This script will do the following operations in order:
Scan a folder
create new calculation folder
Loop the following actions for each homer script


    Unseal homer template file (.homer -> .zip)
    
    
    Unpack zip file (.zip -> normal directory)
    
    
    Open 'problem' file
    
    
    ####### NOT FOR THIS SCRIPT modify 'problem' file
    
    
    Repack zip file (normal directory -> .zip)
    
    
    Reseal homer file (.zip -> .homer) 
        ->[FOR THIS STEP TO FUNCTION PROPERLY, THE JUPYTER NOTEBOOK MUST BE EXECUTED ONE DIRECTORY HIGHER]
    
    
    store updated input file in calculation folder
    
    
    end 

In [11]:
# Script initialization, make sure to set working directory
import os
import numpy as np 
import zipfile
import shutil
import zlib
import pathlib 
import subprocess
from lxml import etree, objectify
folder = 'd:/repo/homer/ClimateStudies/ProductionCases/MinimumLoadRatio/60/16000'

In [12]:
# Unsealing HOMER template files (.homer -> .zip)
files = [f for f in os.listdir(folder) if f.endswith('.homer')]
n_files = np.size(files)
display(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    os.rename(file, base + '.zip')

[]

In [13]:
# Unpack zip file (.zip -> normal directory)
files = [f for f in os.listdir(folder) if f.endswith('.zip')]
n_files = np.size(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(base + 'TEMP')
    os.remove(file)
# Opening each individiual 'problem' file after extraction
    

In [14]:
# (Within normal directory) Edit the .xml files for each HOMER problem.
files = [f for f in os.listdir(folder) if f.endswith('TEMP')]
display(files)
# paths to change as key, new value as value
new_data = {'/HomerProblem/FuelResources/Fuel/Name': 'CCM FUEL',
            '/HomerProblem/FuelResources/Fuel/FuelCostSensitivity/decimal': '0.00',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Initial': '16000000',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Quantity': '1000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/CapitalCostDetail/CostCalculatorItem/UnitCost': '16000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Replacement': '3550000',
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/Quantity': '1000' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/ReplacementCostDetail/CostCalculatorItem/UnitCost': '3550' ,
            '/HomerProblem/Generators/Generator/LifetimeSensitivity/decimal': '87660' ,
            '/HomerProblem/Generators/Generator/CostMatrix/Cost/Operating': '33.42',
            '/HomerProblem/SimulationParameters/Economics/ProjectLifetimeSensitivity/decimal': '60',
            '/HomerProblem/SimulationParameters/SystemControl/Optimizer/FocusFactor': '2',
            '/HomerProblem/Generators/Generator/MinimumLoadRatioSensitivity/decimal': '60'}
   
for i in range(len(files)):
    file = files[i]
    problems = etree.parse(file + '/problem')
    #os.remove(file + '/calculated-problem') #deletes old homer savestate
    for path, value in new_data.items():
       # if 'CapacityOptimization' in path:#not necessary currently, but can be used to differentiate values later
       #     problems.xpath(path)[0].text = value
       #     newfile = etree.tostring(problems,encoding='utf-8',method='xml', xml_declaration='<?xml version="1.0" encoding="utf-8"?>')
       #     newfile = str(newfile,'utf-8')#otherwise use utf-8
       # else: 
            #print(path, ' => ', problems.xpath(path)[0].text)
        problems.xpath(path)[0].text = value
        newfile = etree.tostring(problems,encoding='US-ASCII',method='xml', xml_declaration='<?xml version="1.0" encoding="utf-8"?>')
        newfile = str(newfile,'US-ASCII')#otherwise use utf-8
    with open(file + '/problem','w') as f:
    #   # newfile = etree.tostring(new)
         f.write(newfile)
        
        
        
    #display(problem)
       # problem.xpath(j)[0].text = value[j]

['Case3MLR6050AnchorageDfcTEMP',
 'Case3MLR6050AshlandDfbTEMP',
 'Case3MLR6050AtlantaGACfaTEMP',
 'Case3MLR6050BangorMEDfbTEMP',
 'Case3MLR6050BendOreBskTEMP',
 'Case3MLR6050BentonCountyOregonCsbTEMP',
 'Case3MLR6050BirminghamALCfaTEMP',
 'Case3MLR6050BismarckNDDfbTEMP',
 'Case3MLR6050BlaineCountyMontanaBSkTEMP',
 'Case3MLR6050BoiseIDBSkTEMP',
 'Case3MLR6050BrewsterMADfbTEMP',
 'Case3MLR6050BurlingtonVTDfaTEMP',
 'Case3MLR6050ButtonwillowBwkTEMP',
 'Case3MLR6050CarsonCityNVBskTEMP',
 'Case3MLR6050CheyenneWYBskTEMP',
 'Case3MLR6050ChicagoDfaTEMP',
 'Case3MLR6050CoalvilleUTDfbTEMP',
 'Case3MLR6050ColumbiaMOCfaTEMP',
 'Case3MLR6050ColumbusOHDfaTEMP',
 'Case3MLR6050CrestedButteCODfcTEMP',
 'Case3MLR6050DesMoinesIADfaTEMP',
 'Case3MLR6050DetroitMIDfaTEMP',
 'Case3MLR6050EielsonAFBDfCTEMP',
 'Case3MLR6050EvergladesAMTEMP',
 'Case3MLR6050FlagstaffDsbTEMP',
 'Case3MLR6050GreatFallsMTBskTEMP',
 'Case3MLR6050HiloAfTEMP',
 'Case3MLR6050IdahoFallsDfbTEMP',
 'Case3MLR6050JacksonMSCfaTEMP',
 'Case3M

In [15]:
#Repack zip file (normal directory -> .zip)
## THIS STEP CURRENTLY BREAKS THE HOMER FILE
## Try using zipcode compression 
files = [f for f in os.listdir(folder) if f.endswith('TEMP')]
display(files)
for i in range(len(files)):
    file = files[i]
    directory = pathlib.Path(file)
    basefile = file.replace('TEMP','')
    basefile = basefile.replace('Case3MLR6050','Case3MLR60')
    subprocess.run(["powershell.exe", "Compress-Archive -Path " + file + "\* -DestinationPath " + basefile + " -Force"])
    shutil.rmtree(file) #deletes old temp folders
    

['Case3MLR6050AnchorageDfcTEMP',
 'Case3MLR6050AshlandDfbTEMP',
 'Case3MLR6050AtlantaGACfaTEMP',
 'Case3MLR6050BangorMEDfbTEMP',
 'Case3MLR6050BendOreBskTEMP',
 'Case3MLR6050BentonCountyOregonCsbTEMP',
 'Case3MLR6050BirminghamALCfaTEMP',
 'Case3MLR6050BismarckNDDfbTEMP',
 'Case3MLR6050BlaineCountyMontanaBSkTEMP',
 'Case3MLR6050BoiseIDBSkTEMP',
 'Case3MLR6050BrewsterMADfbTEMP',
 'Case3MLR6050BurlingtonVTDfaTEMP',
 'Case3MLR6050ButtonwillowBwkTEMP',
 'Case3MLR6050CarsonCityNVBskTEMP',
 'Case3MLR6050CheyenneWYBskTEMP',
 'Case3MLR6050ChicagoDfaTEMP',
 'Case3MLR6050CoalvilleUTDfbTEMP',
 'Case3MLR6050ColumbiaMOCfaTEMP',
 'Case3MLR6050ColumbusOHDfaTEMP',
 'Case3MLR6050CrestedButteCODfcTEMP',
 'Case3MLR6050DesMoinesIADfaTEMP',
 'Case3MLR6050DetroitMIDfaTEMP',
 'Case3MLR6050EielsonAFBDfCTEMP',
 'Case3MLR6050EvergladesAMTEMP',
 'Case3MLR6050FlagstaffDsbTEMP',
 'Case3MLR6050GreatFallsMTBskTEMP',
 'Case3MLR6050HiloAfTEMP',
 'Case3MLR6050IdahoFallsDfbTEMP',
 'Case3MLR6050JacksonMSCfaTEMP',
 'Case3M

In [16]:
# Reseal homer file (.zip -> .homer)
files = [f for f in os.listdir(folder) if f.endswith('.zip')]
n_files = np.size(files)
type(files)
for i in range(len(files)):
    file = files[i]
    base = os.path.splitext(file)[0]
    os.rename(file, base + '.homer')